In [10]:
#importação das bibliotecas iniciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline 

In [14]:
#carregamento da base de dados da usina
geracao = pd.read_csv('dados\Plant_1_Generation_Data.csv')
clima = pd.read_csv('dados\Plant_1_Weather_Sensor_Data.csv')

In [16]:
#escopo geral da base de dados
geracao.head()
clima.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0


In [17]:
#descrição do comportamento das varáveis dos dados
geracao.describe()
clima.describe()

,PLANT_ID,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
count,3182.0,3182.000000,3182.000000,3182.000000
mean,4135001.0,25.531606,31.091015,0.228313
std,0.0,3.354856,12.261222,0.300836
min,4135001.0,20.398505,18.140415,0.000000
25%,4135001.0,22.705182,21.090553,0.000000
50%,4135001.0,24.613814,24.618060,0.024653
75%,4135001.0,27.920532,41.307840,0.449588
max,4135001.0,35.252486,65.545714,1.221652


In [22]:
a = geracao.isnull().sum()
b = clima.isnull().sum()
print(f'Quantidade de elemesntos nulo da geração:\n{a}')
print(f'Quantidade de elemesntos nulo ddo clima:\n{b}')

Quantidade de elemesntos nulo da geração:
DATE_TIME      0
PLANT_ID       0
SOURCE_KEY     0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
dtype: int64
Quantidade de elemesntos nulo ddo clima:
DATE_TIME              0
PLANT_ID               0
SOURCE_KEY             0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64


In [23]:
#Convertendo as datas para DateTime
geracao['DATE_TIME'] = pd.to_datetime(geracao['DATE_TIME'],format = '%d-%m-%Y %H:%M')
clima['DATE_TIME'] = pd.to_datetime(clima['DATE_TIME'],format = '%Y-%m-%d %H:%M')

In [25]:
#criando colunas para tempo e data
geracao['DATE'] = geracao['DATE_TIME'].apply(lambda x:x.date())
geracao['TIME'] = geracao['DATE_TIME'].apply(lambda x:x.time())
clima['DATE'] = clima['DATE_TIME'].apply(lambda x:x.date())
clima['TIME'] = clima['DATE_TIME'].apply(lambda x:x.time())